In [1]:
import os, time, json, requests, pandas as pd
from pathlib import Path

In [8]:
# ── config ─────────────────────────────────────────────────────────────
API_URL      = "https://services.nvd.nist.gov/rest/json/cves/2.0"
API_KEY      = os.getenv("NVD_API_KEY") or "ea5501a5-24fe-4720-80e3-2abed401d92f"
whitelist    = pd.read_csv("../data/whitelist1.csv")   # your cleaned file
cpe_uri      = whitelist['cpeName'].tolist()

OUTFILE      = Path("../data/smb_cves.parquet")
RATE_SECS    = 1.2                    # stay below 5-sec soft limit
PER_PAGE     = 2000                   # max allowed by v2.0
# ───────────────────────────────────────────────────────────────────────


headers   = {"apiKey": API_KEY}
start     = 0
all_items = []

while True:
    params = {
        "cpeName": cpe_uri,         # exact-match search
        "resultsPerPage": PER_PAGE,
        "startIndex":   start
    }
    r = requests.get(API_URL, headers=headers, params=params, timeout=20)
    r.raise_for_status()            # raise if HTTP error (4xx / 5xx)

    data      = r.json()
    vulns     = data.get("vulnerabilities", [])
    all_items.extend(vulns)

    # stop when we’ve read the last page
    start += PER_PAGE
    if start >= data.get("totalResults", 0) or not vulns:
        break
    time.sleep(RATE_SECS)

print(f"✅ Retrieved {len(all_items)} CVEs for Adobe Acrobat")

# ── flatten each CVE into a one-row dict ───────────────────────────────
def extract_fields(vuln: dict) -> dict:
    cve      = vuln["cve"]
    metrics  = cve.get("metrics", {})
    refs     = cve.get("references", [])

    # Prefer CVSS v3.1, fall back to v3.0
    cvss31   = metrics.get("cvssMetricV31", [{}])[0].get("cvssData", {})
    cvss30   = metrics.get("cvssMetricV30", [{}])[0].get("cvssData", {})

    cvss     = cvss31 or cvss30                          # whichever exists

    # Grab first English description
    descr    = next(
        (d["value"] for d in cve.get("descriptions", []) if d["lang"] == "en"),
        ""
    )

    # Collect CWE IDs (could be many)
    cwes = [
        w["value"]
        for w in cve.get("weaknesses", [])
        for wmeta in w.get("description", [])
        for w in [wmeta]
        if w.get("lang") == "en"
    ]

    return {
        "cve_id":        cve["id"],
        "published":     cve.get("published"),
        "last_modified": cve.get("lastModified"),

        "cvss_base":     cvss.get("baseScore"),
        "severity":      cvss.get("baseSeverity"),
        "attack_vector": cvss.get("attackVector"),
        "attack_complexity": cvss.get("attackComplexity"),
        "priv_required": cvss.get("privilegesRequired"),
        "user_interact": cvss.get("userInteraction"),

        "cwes":          ";".join(cwes) if cwes else None,
        "description":   descr[:500],           # trim super-long text

        # keep a pipe-separated list of reference URLs
        "references":    "|".join(r["url"] for r in refs[:10]),

        "full_json":     vuln                   # raw blob for later GPT work
    }

# Build DataFrame
records = [extract_fields(v) for v in all_items]
df      = pd.DataFrame(records)

# peek
df.head()

HTTPError: 414 Client Error: Request-URI Too Large for url: https://services.nvd.nist.gov/rest/json/cves/2.0?cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A1%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A10.0.1%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A10.0.1%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A10.0.2%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A10.0.3%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A10%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A10%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&cpeName=cpe%3A2.3%3Aa%3Aadobe%3Aacrobat%3A10.1.0%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A%3A%2A&

In [5]:
# ingest_all_cves.ipynb  –  pull every CVE for every CPE in the whitelist
import os, time, requests, pandas as pd
from pathlib import Path

# ── config ─────────────────────────────────────────────────────────────
API_URL      = "https://services.nvd.nist.gov/rest/json/cves/2.0"
API_KEY      = os.getenv("NVD_API_KEY") or "ea5501a5-24fe-4720-80e3-2abed401d92f"
WHITELIST    = Path("../data/whitelist1.csv")
OUTFILE      = Path("../data/cves1.parquet")
RATE_SECS    = 1.0
PER_PAGE     = 2000
PROGRESS_EVERY = 25                # how often to print a status line
# ───────────────────────────────────────────────────────────────────────

def fetch_cves_for_cpe(cpe_uri: str) -> list[dict]:
    parts = cpe_uri.split(":")
    if len(parts) < 6:
        return []
    cpe_query = ":".join(parts[:6]) if parts[5] == "*" else cpe_uri

    all_items, start = [], 0
    headers = {"apiKey": API_KEY}

    while True:
        params = {
            "cpeName":        cpe_query,
            "resultsPerPage": PER_PAGE,
            "startIndex":     start,
            "pubStartDate": f"{start_date}T00:00:00.000Z",
            "pubEndDate": f"{end_date}T23:59:59.999Z",
        }
        r = requests.get(API_URL, headers=headers, params=params, timeout=30)
        if r.status_code != 200:
            print(f"⚠️ {cpe_query[:70]} → {r.status_code}")
            break

        data   = r.json()
        items  = data.get("vulnerabilities", [])
        all_items.extend(items)

        start += PER_PAGE
        if start >= data.get("totalResults", 0) or not items:
            break
        time.sleep(RATE_SECS)
    return all_items

def flatten(v: dict, cpe_uri: str) -> dict:
    cve      = v["cve"]
    metrics  = cve.get("metrics", {})
    cvss31   = metrics.get("cvssMetricV31", [{}])[0].get("cvssData", {})
    cvss30   = metrics.get("cvssMetricV30", [{}])[0].get("cvssData", {})
    cvss     = cvss31 or cvss30
    descr    = next((d["value"] for d in cve.get("descriptions", []) if d["lang"] == "en"), "")
    cwes     = [
        d["value"] for w in cve.get("weaknesses", [])
        for d in w.get("description", []) if d.get("lang") == "en"
    ]
    refs     = "|".join(r["url"] for r in cve.get("references", [])[:10])

    return {
        "cve_id":            cve["id"],
        "cpe_name":          cpe_uri,
        "published":         cve.get("published"),
        "last_modified":     cve.get("lastModified"),

        "cvss_base":         cvss.get("baseScore"),
        "severity":          cvss.get("baseSeverity"),
        "attack_vector":     cvss.get("attackVector"),
        "attack_complexity": cvss.get("attackComplexity"),
        "priv_required":     cvss.get("privilegesRequired"),
        "user_interact":     cvss.get("userInteraction"),

        "cwes":              ";".join(cwes) if cwes else None,
        "description":       descr[:1000],
        "references":        refs,
        "full_json":         v,
    }

# ── 1. load whitelist ─────────────────────────────────────────────────
cpe_list = (
    pd.read_csv(WHITELIST, dtype=str)["cpeName"]
      .dropna()
      .unique()
)
print(f"📋  {len(cpe_list):,} unique CPEs to query")

# ── 2. query API ───────────────────────────────────────────────────────
rows = []
for idx, cpe in enumerate(cpe_list, start=1):
    if idx % PROGRESS_EVERY == 0 or idx == 1:
        print(f"  → {idx}/{len(cpe_list)}   {cpe[:70]}…")

    for vuln in fetch_cves_for_cpe(cpe):
        rows.append(flatten(vuln, cpe))

print("✔️  API queries finished")

# ── 3. build DataFrame & de-dup ────────────────────────────────────────
df = (
    pd.DataFrame(rows)
      .drop_duplicates(subset=["cve_id", "cpe_name"])
      .reset_index(drop=True)
)
print(f"🗂  {df.shape[0]:,} CVE–CPE rows collected")

📋  1,000 unique CPEs to query
  → 1/1000   cpe:2.3:a:adobe:acrobat:-:*:*:*:*:*:*:*…


KeyboardInterrupt: 

In [ ]:
# ── 4. save ────────────────────────────────────────────────────────────
df.to_parquet(OUTFILE, index=False)
print(f"✅  Saved to {OUTFILE.resolve()}")